In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
%cd ..

c:\Users\heroe\OneDrive\桌面\play-gpt\vnov


C:\Users\heroe\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import json
from vnov.data import Novel
from vnov.image_generation.midjourney import Midjourney
import threading
from concurrent.futures import ThreadPoolExecutor
import time
import random
# from collections import deque
# from vnov.utils.rate_limiter import RateLimiter

In [5]:
with open('./datasets/yujie/prompts/eng_director_prompts_Midjourney.json', "r", encoding="utf-8") as f:
    mj_prompts = json.load(f)
mj_prompts[:5]

['A woman stands outdoors on a cloudy, cold day, with a bleak landscape of barren hills, a funeral hall, crematorium, withered trees, and tombstones in the background. She is wearing a neat black long dress, an open black coat, and simple black gloves. She stands with her hands clasped in front of her, slightly bowing her head, staring coldly at the funeral hall.',
 'A woman stands outdoors on a sunny day with sunlight filtering through the clouds and a gentle breeze. She is wearing a neat black long dress and an open black coat. She has on simple black gloves. She stands near a desolate mountain, a spirit hall, a crematorium, and tombstones, looking upward with an expression of pride and disdain.',
 "A man stands outdoors in a bleak, overcast environment with biting winds. He is dressed in a neatly pressed dark suit and an open black trench coat. On his wrist, he wears a luxurious silver watch. The scene is set in a desolate area with a dilapidated mausoleum, a crematorium, and numero

In [6]:
with open("./datasets/yujie/storyboard/storyboard_combined.json", "r", encoding="utf-8") as f:
    storyboard = json.load(f)
storyboard[0]

{'原文起始点': '魔都四十公里外，一片清冷的荒山，葬礼的气氛显得异常冷清，除了两名火葬场的工作人员，我和陈金莲站在这里，面临着我人生的终结。',
 '场景': {'位置类型': '户外',
  '环境特点': ['阴云密布', '寒风刺骨', '白天'],
  '场景特征': ['荒山', '灵堂', '火葬场', '枯树', '墓碑']},
 '幕': 0,
 '角色': [{'名字': '陈金莲',
   '动作': ['站在灵堂前，双手合拢，微微低头'],
   '视线与方向': '凝视前方的灵堂',
   '表情': ['冷漠', '无情'],
   '服装': {'上衣': {'物品': '黑色长裙', '状态': '整洁'},
    '外套': {'物品': '黑色大衣', '状态': '敞开'},
    '配饰': {'物品': '黑色手套', '类型': '简约款'}}}]}

In [7]:
assert(len(storyboard) == len(mj_prompts))

In [8]:
with open("./datasets/yujie/mj_char_images/char_to_url.json", "r", encoding="utf-8") as f:
    char_to_url = json.load(f)
char_to_url

{'陈金莲': 'https://s.mj.run/2wejRultpjc',
 '秦霜': 'https://s.mj.run/cdulSXPnpAM',
 '我': 'https://s.mj.run/DM_oYmfiHWw',
 '阿东': 'https://s.mj.run/X6G_2x2ueTU',
 '马蓉': 'https://s.mj.run/NvumizMSDGk',
 '杨自奎': 'https://s.mj.run/pn164xzRMK8',
 '阮俊': 'https://s.mj.run/ol19ChMx0Mc',
 '代红伟': 'https://s.mj.run/Dj2Q4tZDvKs',
 '孙小萌': 'https://s.mj.run/hzXvW5VfjpE',
 '徐梦娜': 'https://s.mj.run/unt0frbx5BI',
 '吴龙飞': 'https://s.mj.run/tL1FCiHX_mg',
 '黄三': 'https://s.mj.run/SGWoM5IAAFY',
 '肖雅': 'https://s.mj.run/5hvApCTDr_M',
 '黎莹': 'https://s.mj.run/-8TYa_SYsG8',
 '德爷': 'https://s.mj.run/m9qh9zyeKG0',
 '钱九': 'https://s.mj.run/K4WQ9t0jnGQ',
 '江河': 'https://s.mj.run/QiJEzkf-Xz0',
 '赖三': 'https://s.mj.run/YUB5NVyuOfo'}

In [9]:
# scene_to_url = [None] * len(mj_prompts)
# for i, prompt in enumerate(mj_prompts):
#     scene_info = storyboard[i]
#     characters = scene_info['角色']
#     characters = [item['名字'] for item in characters]
#     if len(characters) == 1 and characters[0] in char_to_url:
#         char_url = char_to_url[characters[0]]
#         prompt = prompt + f' --cref {char_url}'
#     prompt += f' --sref {char_url} --niji 6 --style raw --sw 55'
#     url = mj.fetch_image(prompt, './datasets/yujie/scene_images', f'scene{i}.png')
#     scene_to_url[i] = url

In [10]:
with open("./datasets/yujie/scene_images2/scene_to_url.json", "r", encoding="utf-8") as f:
    scene_to_url = json.load(f)
scene_to_url
# scene_to_url = [None for _ in storyboard]

[['https://media.discordapp.net/attachments/1269511325181022229/1296974964364410910/uklu7405_anime_episode_still_swarm_a_woman_stands_outdoors_on_a_c405412d-64be-4f7c-9417-e67ea43efed3.png?ex=67143d90&is=6712ec10&hm=ef0e5b923d91340561912465fb5ca41c8b5e15a8ee1e897a1ab8ef22524cf84c&',
  'https://media.discordapp.net/attachments/1269511325181022229/1296975032710332426/uklu7405_anime_episode_still_swarm_a_woman_stands_outdoors_on_a_2c4f59e3-0a77-4a78-8f60-a1627e59392c.png?ex=67143da0&is=6712ec20&hm=8289a3c4f99d81be41d165957d75c7c240b4cfb7ace36cd46edd4db780a03d0b&',
  'https://media.discordapp.net/attachments/1269511325181022229/1296975076553527366/uklu7405_anime_episode_still_swarm_a_woman_stands_outdoors_on_a_516f8c89-5580-48f9-8bf6-37beac8be4ff.png?ex=67143daa&is=6712ec2a&hm=f2ae2024aba89653b55feb38a92c23a21863d1704bea348eddad35ab19d18123&',
  'https://media.discordapp.net/attachments/1269511325181022229/1296975119112999016/uklu7405_anime_episode_still_swarm_a_woman_stands_outdoors_on_a_

TODO: abstract away clean prompt and cref etc

TODO: 4 images download all, add multicore

In [11]:
taotao = "https://s.mj.run/qRhIa9TaD5Q"

In [12]:
char_to_url_lock = threading.Lock()
scene_to_url_lock = threading.Lock()

def process_scene(i):
    # print("start", i)
    if scene_to_url[i] is not None:
        return
    mj = Midjourney()
    # time.sleep(random.uniform(1, 10))
    prompt = mj_prompts[i]
    scene_info = storyboard[i]
    characters = scene_info['角色']
    characters = [item['名字'] for item in characters]
    prompt = "anime episode still image: " + prompt + ", anime, japanese manga style. Art by Kohei Horikoshi."
    print("setup", i)

    if len(characters) == 1 and characters[0] in char_to_url:
        char_url = char_to_url[characters[0]]
        if not char_url.startswith("https://s.mj.run/"):
            # print("prepare to fetch", i)
            urls = mj.fetch_image(prompt, image_folder='./datasets/yujie/scene_images2', image_name=f'scene{i}', cref=char_url, cw=20, sref=taotao, sw=45, niji=True, need_new_ref_url=True)
            # print("waiting for lock", i)
            with char_to_url_lock:
                print("in char lock", i)
                char_to_url[characters[0]] = urls['cref_url']
                print(f"Updated char_to_url for character: {characters[0]}")
        else:
            # print("prepare to fetch", i)
            urls = mj.fetch_image(prompt, image_folder='./datasets/yujie/scene_images2', image_name=f'scene{i}', cref=char_url, cw=20, sref=taotao, niji=True, sw=45)
        url = urls['image_url']
    else:
        # print("prepare to fetch", i)
        urls = mj.fetch_image(prompt, image_folder='./datasets/yujie/scene_images2', image_name=f'scene{i}', sref=taotao, niji=True, sw=200)
        url = urls['image_url']

    with scene_to_url_lock:
        scene_to_url[i] = url
    print(f"Processed scene {i}")

    # Save progress every 3 scenes
    if i % 5 == 0:
        with scene_to_url_lock:
            with open("./datasets/yujie/scene_images2/scene_to_url.json", "w", encoding="utf-8") as f:
                json.dump(scene_to_url, f, indent=4, ensure_ascii=False)
            print(f"Saved progress at scene {i}")

In [ ]:
with ThreadPoolExecutor(max_workers=4) as executor:
    # Submit tasks for each scene=
    executor.map(process_scene, range(1, len(scene_to_url)))

setup 139
setup 148
setup 529
deque([1729383150.2484996])
anime episode still image: a dimly lit indoor room with loud music playing, featuring a private booth with a sofa, scattered wine bottles, and mood lighting. a woman named qin shuang stands in the center, wearing a sleek black fitted jacket and a clean white short skirt, paired with neatly worn high heels. her expression is one of surprise and confusion, with slightly raised eyebrows and a slight frown. she has her hands crossed over her  and is looking directly at the viewer., anime, japanese manga style. art by kohei horikoshi. --cref https://s.mj.run/cdulSXPnpAM --sref https://s.mj.run/qRhIa9TaD5Q --sw 45--cw 20 --niji 6 +sent and respond+ <Response [429]>
deque([1729383150.2484996])
anime episode still image: in a dimly lit, music-filled indoor setting, a private room features a sofa, scattered bottles, and mood lighting. a woman named qin shuang stands up, gently wiping  from the face of another person. she is dressed in a 

In [13]:
scene_to_url.count(None)

2

In [21]:
with open("./datasets/yujie/scene_images2/scene_to_url.json", "w", encoding="utf-8") as f:
    json.dump(scene_to_url, f, indent = 4, ensure_ascii=False)

In [22]:
with open("./datasets/yujie/mj_char_images/char_to_url.json", "w", encoding="utf-8") as f:
    json.dump(char_to_url, f, indent = 4, ensure_ascii=False)

In [23]:
with open("./datasets/yujie/scene_images2/scene_to_url.json", "r", encoding="utf-8") as f:
    scene_to_url = json.load(f)
scene_to_url

[['https://media.discordapp.net/attachments/1269511325181022229/1296974964364410910/uklu7405_anime_episode_still_swarm_a_woman_stands_outdoors_on_a_c405412d-64be-4f7c-9417-e67ea43efed3.png?ex=67143d90&is=6712ec10&hm=ef0e5b923d91340561912465fb5ca41c8b5e15a8ee1e897a1ab8ef22524cf84c&',
  'https://media.discordapp.net/attachments/1269511325181022229/1296975032710332426/uklu7405_anime_episode_still_swarm_a_woman_stands_outdoors_on_a_2c4f59e3-0a77-4a78-8f60-a1627e59392c.png?ex=67143da0&is=6712ec20&hm=8289a3c4f99d81be41d165957d75c7c240b4cfb7ace36cd46edd4db780a03d0b&',
  'https://media.discordapp.net/attachments/1269511325181022229/1296975076553527366/uklu7405_anime_episode_still_swarm_a_woman_stands_outdoors_on_a_516f8c89-5580-48f9-8bf6-37beac8be4ff.png?ex=67143daa&is=6712ec2a&hm=f2ae2024aba89653b55feb38a92c23a21863d1704bea348eddad35ab19d18123&',
  'https://media.discordapp.net/attachments/1269511325181022229/1296975119112999016/uklu7405_anime_episode_still_swarm_a_woman_stands_outdoors_on_a_

single thread 查漏补缺

In [25]:
# scene_to_url = [None for _ in storyboard]
i = 0

while i < len(scene_to_url):
    if scene_to_url[i] is not None:
        i += 1
        continue
    print("start", i)
    mj = Midjourney()
    prompt = mj_prompts[i]
    scene_info = storyboard[i]
    characters = scene_info['角色']
    characters = [item['名字'] for item in characters]
    prompt = "anime episode still image: " + prompt + ", anime, japanese manga style"
    if len(characters) == 1 and characters[0] in char_to_url:
        char_url = char_to_url[characters[0]]
        if not char_url.startswith("https://s.mj.run/"):
            urls = mj.fetch_image(prompt, './datasets/yujie/scene_images2', f'scene{i}.png', cref=char_url, cw=20, sref=char_url, sw=45, niji=True, need_new_ref_url=True)
            char_to_url[characters[0]] = urls['cref_url']
            print("here")
        else:
            urls = mj.fetch_image(prompt, './datasets/yujie/scene_images2', f'scene{i}.png', cref=char_url, cw=20, sref=char_url, niji=True, sw=45)
        url = urls['image_url']
    else:
        urls = mj.fetch_image(prompt, './datasets/yujie/scene_images2', f'scene{i}.png', sref=taotao, niji= True, sw=200)
        url = urls['image_url']
    scene_to_url[i] = url
    # print(i)
    if i % 1 == 0:
        with open("./datasets/yujie/scene_images2/scene_to_url.json", "w", encoding="utf-8") as f:
            json.dump(scene_to_url, f, indent=4, ensure_ascii=False)
    i += 1
    break

start 29
starting 0 anime episode still image: a warm and cozy indoor scene in a milk tea shop with soft lighting and a relaxed atmosphere. the setting includes a table, chairs, a glass window, and decorative paintings. a young woman named ma rong is leaning towards another person, showing concern and curiosity. she is wearing a neatly pressed blue shirt and well-fitted black jeans. the scene captures her attentive expression as she gazes at the other person., anime, japanese manga style
prompt: anime episode still image: a warm and cozy indoor scene in a milk tea shop with soft lighting and a relaxed atmosphere. the setting includes a table, chairs, a glass window, and decorative paintings. a young woman named ma rong is leaning towards another person, showing concern and curiosity. she is wearing a neatly pressed blue shirt and well-fitted black jeans. the scene captures her attentive expression as she gazes at the other person., anime, japanese manga style
matches: anime episode sti